# Introduction
On this notebook I will do several things on the mushroom dataset:

1. Run a simple logistic regression
2. Fit a model using random forest classifier
3. Asses the feature importance, and reduce the number of features accordingly
4. Do some visualization

Before I fit the model using logistic regression, I will clean and prepare the data first by using one-hot encoder.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
shroom = pd.read_csv('../input/mushrooms.csv')
shroom[:3]

In [ ]:
for feature in shroom.columns:
    print(feature, ':', shroom[feature].unique())

It appears that all have categorical values. I think we need to change them into numerical values. But before that I think I will remove the missing values first, which occurs on the variable 'stalk-root'. I will also drop the 'veil-type' feature since it only consists of one value.

In [ ]:
shroom = shroom.drop(shroom[shroom['stalk-root']=='?'].index)
shroom = shroom.drop('veil-type', axis=1)

In [ ]:
# making sure no other missing values
shroom.isnull().sum().sum()

For features with two values I will use `LabelBinarizer`, otherwise I will use `pd.get_dummies`.

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
for feature in shroom.columns:
    if len(shroom[feature].unique()) == 2:
        shroom[feature] = lb.fit_transform(shroom[feature])

In [ ]:
features_onehot = []
for feature in shroom.columns[1:]:
    if len(shroom[feature].unique()) > 2:
        features_onehot.append(feature)
temp = pd.get_dummies(shroom[features_onehot])
shroom = shroom.join(temp)
shroom = shroom.drop(features_onehot, axis=1)
shroom[:3]

In [ ]:
# making sure all values are numerical
np.unique(shroom.values)

In [ ]:
from sklearn.model_selection import train_test_split

X = shroom.drop('class', axis=1).values
y = shroom['class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
print('X_train Shape:', X_train.shape)
print('X_test Shape:', X_test.shape)
print('y_train Shape:', y_train.shape)
print('y_test Shape:', y_test.shape)

## Logistic Regression
Now we are ready to train the data.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Training Accuracy: %.2f%%' % (lr.score(X_train, y_train)*100))
print('Test Accuracy: %.2f%%' % (lr.score(X_test, y_test)*100))

With 100% accuracy, it is either the data is actually linearly separable or there is something wrong in my code.

# Random Forest
Now I will use random forest classifier to fit the model and to find out the feature importance. Since random forest require no one-hot encoding, I will load the data again. This time I will not drop the 'veil-type' feature just to see what will happen.

In [ ]:
mushroom = pd.read_csv('../input/mushrooms.csv')
shroom2 = mushroom.drop(mushroom[mushroom['stalk-root']=='?'].index)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
lbe = LabelEncoder()
for feature in shroom2.columns[1:]:
    shroom2[feature] = lbe.fit_transform(shroom2[feature])
shroom2[:3]

In [ ]:
y = shroom2['class'].values
X = shroom2.drop('class', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rfc.fit(X_train, y_train)
print('Training Score: %.2f%%' % (rfc.score(X_train, y_train) * 100))
print('Test Score: %.2f%%' % (rfc.score(X_test, y_test) * 100))

In [ ]:
importances = rfc.feature_importances_
features = shroom2.columns[1:]
sort_indices = np.argsort(importances)[::-1]
sorted_features = []
for idx in sort_indices:
    sorted_features.append(features[idx])
plt.figure()
plt.bar(range(len(importances)), importances[sort_indices], align='center');
plt.xticks(range(len(importances)), sorted_features, rotation='vertical');
plt.xlim([-1, len(importances)])
plt.grid(False)

## Dimensionality Reduction
It seems that we can fit the model just using the top features. So we will again fit the model using logistic regression and random forest classifier using just those features and see what will happen.

In [ ]:
# random forest
top_features = sorted_features[:2]
y = shroom2['class'].values
X = shroom2[top_features].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rfc.fit(X_train, y_train)
print('Random Forest with 2 features')
print('Training Score: %.2f%%' % (rfc.score(X_train, y_train) * 100))
print('Test Score: %.2f%%' % (rfc.score(X_test, y_test) * 100))

They looks good enough to me, considering we use much fewer feature than we did previously.

In [ ]:
# logistic regression
top_features_2 = [
    'odor_a', 'odor_c', 'odor_f', 'odor_l', 'odor_m', 'odor_n', 'odor_p', # feature 1
    'spore-print-color_h', 'spore-print-color_k','spore-print-color_n', # feature 2
    'spore-print-color_r', 'spore-print-color_u', 'spore-print-color_w',
]
y = shroom['class'].values
X = shroom[top_features_2].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Logistic Regression with 2 features')
print('Training Accuracy: %.2f%%' % (lr.score(X_train, y_train)*100))
print('Test Accuracy: %.2f%%' % (lr.score(X_test, y_test)*100))

### What if we just use one variable?

In [ ]:
# random forest
top_features = sorted_features[:1]
y = shroom2['class'].values
X = shroom2[top_features].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rfc.fit(X_train, y_train)
print('Random Forest with 1 feature')
print('Training Score: %.2f%%' % (rfc.score(X_train, y_train) * 100))
print('Test Score: %.2f%%' % (rfc.score(X_test, y_test) * 100))

In [ ]:
# logistic regression
top_features_2 = [
    'odor_a', 'odor_c', 'odor_f', 'odor_l', 'odor_m', 'odor_n', 'odor_p', # feature 1
]
y = shroom['class'].values
X = shroom[top_features_2].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Logistic Regression with 1 feature')
print('Training Accuracy: %.2f%%' % (lr.score(X_train, y_train)*100))
print('Test Accuracy: %.2f%%' % (lr.score(X_test, y_test)*100))

It turns out that one feature is pretty good. Next we'll try to do some visualization.

# Data Visualization

In [ ]:
sns.factorplot('class', col='odor', data=mushroom, kind='count', size=2.5, aspect=0.6, col_wrap=5);

In [ ]:
g = sns.factorplot('class', col='spore-print-color', data=mushroom,
               kind='count', size=2.5, aspect=0.6, col_wrap=5)
g.set_titles("{col_name}")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('spore-print-color');

The figures above suggest we can tell the difference between poisonous and edible mushrooms pretty well just by looking at the `odor` and the `spore-print-color`.

### Visualization for feature with low importance
Now we will see the visualization for the feature that have low importance according to random forest.

In [ ]:
sns.factorplot('class', col='gill-attachment', data=mushroom, kind='count', size=2.5, aspect=0.6, col_wrap=5);

You can see that most mushrooms, either poisonous or not, have `gill-attachment=f`, so it's not useful as a predictor.

Enough for now. Thanks for your attention.